In [ ]:
%matplotlib inline
from PIL import Image
import tensorflow as tf
import numpy as np
import time
import sys
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
sys.path.append("/project/train/src_repo/tf-models/research")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils


colors=["#112233","#ce0000","#FF0080","#5B5B00","#796400"]


PATH_TO_LABELS="/project/train/cvmart/dataset/label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,  use_display_name=True)
  #train/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/frozen_inference_graph.pb
  
#model_pb_path = "/project/train/cvmart/models/final/ssd_inception_v2.pb"
model_pb_path = "/project/train/cvmart/models/final/ssd_inception_v2.pb"
if not os.path.isfile(model_pb_path):
    log.error(f'{model_pb_path} does not exist')
else:

  detection_graph = tf.Graph()
  with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(model_pb_path, 'rb') as fid:
          serialized_graph = fid.read()
          od_graph_def.ParseFromString(serialized_graph)
          tf.import_graph_def(od_graph_def, name='')

  print('Initializing session...')
  #global sess
  sess = tf.Session(graph=detection_graph)
  im = Image.open("/project/train/cvmart/dataset/image/0_246.jpg")
  im_w=im.size[0]
  im_h=im.size[1]
  image_np_with_detections = im.copy()    
  input_image = np.expand_dims(im, axis=0)
  net=detection_graph
  # --------------------------- Performing inference ----------------------------------
  # Extract image tensor
  image_tensor = net.get_tensor_by_name('image_tensor:0')
  # Extract detection boxes, scores, classes, number of detections
  boxes = net.get_tensor_by_name('detection_boxes:0')
  scores = net.get_tensor_by_name('detection_scores:0')
  classes = net.get_tensor_by_name('detection_classes:0')
  num_detections = net.get_tensor_by_name('num_detections:0')
  # Actual detection.
  (boxes, scores, classes, num_detections) = sess.run(
    [boxes, scores, classes, num_detections],
    feed_dict={image_tensor: input_image})
  num_detections=int(num_detections[0])
  print(num_detections)
  plt.imshow(im)

  for i in range(num_detections):
      box= boxes[0][i]
      print(scores[0][i],box)
      if(scores[0][i]>0.22):
        xmin = int(float(box[1])*im_w )
        ymin = int(float(box[0])*im_h )
        xmax = int(float(box[3])*im_w )
        ymax = int(float(box[2])*im_h )
        cls_id=int(classes[0][i])
        print(cls_id)
        #print(colors[cls_id])
        rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                              ymax - ymin, fill=False,
                              edgecolor=colors[0],
                              linewidth=2)
        plt.gca().add_patch(rect)

  plt.figure(figsize=(40,40))
  #plt.imshow(image_np_with_detections)
  plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
colors=["#112233","#ce0000","#FF0080","#5B5B00","#796400"]
labels={"lifevest":1,"lifevest_person":2,"no_lifevest_person":3,"no_lifevest":4}
DATA_SETPATH="/project/train/cvmart/dataset/image"
def get_XML_PATHS(path_str):
  XML_PATHS=[]
  paths = os.listdir(path_str)
  for p in paths:
    if(p[-4:]==".xml"):
      XML_PATHS.append(os.path.join(path_str,p))
  return XML_PATHS
def get_IMG_PATHS(path_str):
  IMG_PATHS=[]
  paths = os.listdir(path_str)
  for p in paths:
    lowstr=p[-4:].lower()
    if(lowstr==".jpg" or lowstr==".png"):
        IMG_PATHS.append(os.path.join(path_str,p))
  return IMG_PATHS

def plt_bboxes(img, boxs,classes ):
    """Visualize bounding boxes. Largely inspired by SSD-MXNET!
    """

    plt.imshow(img)

    for i,box in enumerate(boxs):
   
      xmin = int(box[0] )
      ymin = int(box[1] )
      xmax = int(box[2] )
      ymax = int(box[3] )
      cls_id=classes[i]
      #print(colors[cls_id])
      rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                            ymax - ymin, fill=False,
                            edgecolor=colors[cls_id],
                            linewidth=2)
      class_name=str(cls_id)
      plt.gca().add_patch(rect)
      plt.gca().text(xmin, ymin - 2,
                 '{:s} '.format(class_name),
                      bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                      fontsize=12, color='white')
def parse_xml(name):
  tree = ET.parse(name)
  root = tree.getroot()
  boxs=[]
  classes=[]
  for member in root.findall('object'):
    
    xmin  = int(float(member[4][0].text))
    ymin  = int(float(member[4][1].text) )
    xmax  = int(float(member[4][2].text)) 
    ymax  = int(float(member[4][3].text))
    print(xmin,ymin,xmax,ymax)
    print(member[0].text)
    cls=labels[member[0].text]
    #print(cls)
    boxs.append((xmin,ymin,xmax,ymax))
    classes.append(cls)
  return boxs,classes




IMAGE_PATHS=get_IMG_PATHS(DATA_SETPATH)

for name in IMAGE_PATHS:


    xmlname= name[0:-4]+".xml"
    imgname= name[0:-4]+".jpg"
    print(xmlname)
    h=w=3
    boxlist ,classes=parse_xml(xmlname)
    plt.figure(figsize=(20,20))
    im = Image.open(name)
    # im,imglist=image_cut(imgname,h,w)
    plt_bboxes(np.array(im),boxlist,classes)
    plt.show()

# #crop,boxlist,classes,box_w,box_h=box_cut(xmlname,h,w)
# print(len(imglist),len(boxlist),len(classes))
# for i,im in enumerate(imglist):
#   plt_bboxes(np.array(im),boxlist[i],classes[i])
#   plt.show()

In [ ]:
import os
import xml.etree.ElementTree as ET
def test_xml(name):
    tree = ET.parse(name)
    root = tree.getroot()
    boxs=[]
    classes=[]
    v1=v2=v3=v4=0
    for member in root.findall('object'):
        if(member[0].text=="lifevest"):
            v1=v1+1
        elif(member[0].text=="lifevest_person"):
            v2=v2+1 
        elif(member[0].text=="no_lifevest_person"):
            v3=v3+1
        elif(member[0].text=="no_lifevest"):
            v4=v4+1
    if(v1!=v2):
        print("error 1",name)
    if(v3!=v4):
        print("error 2",name)  
        
DATA_SETPATH="/project/train/NJSK"
def get_XML_PATHS(path_str):
  XML_PATHS=[]
  paths = os.listdir(path_str)
  for p in paths:
    if(p[-4:]==".xml"):
      #XML_PATHS.append(os.path.join(path_str,p))
        XML_PATHS.append(p)
  return XML_PATHS
paths= get_XML_PATHS(DATA_SETPATH)
p2= os.listdir(DATA_SETPATH)
for p in paths:
    #test_xml(p)
    if(p in p2):
        pass
    else:
        print(p)

In [ ]:
alist=[]
blist=[]
clist=[]
dlist=[]
count=0
with open("/project/train/1.txt","r") as f:

    for line in f.readlines():                          #依次读取每行  
        line = line.strip()                             #去掉每行头尾空白  
        a=line.split(":")
        if(len(alist)==0):
            alist.append(a[0])
            blist.append(1)
            count=1
        else:
            if(a[0] in alist):
                blist[count-1]=blist[count-1]+1
            else:
                alist.append(a[0])
                blist.append(1)
                count=count+1
count=0
with open("/project/train/2.txt","r") as f:

    for line in f.readlines():                          #依次读取每行  
        line = line.strip()                             #去掉每行头尾空白  
        a=line.split(":")
        if(len(clist)==0):
            clist.append(a[0])
            dlist.append(1)
            count=1
        else:
            if(a[0] in clist):
                dlist[count-1]=dlist[count-1]+1
            else:
                clist.append(a[0])
                dlist.append(1)
                count=count+1
                
    for i, line in enumerate(alist):
        if(blist[i]==2):
            #print(line,blist[i])
            #print(line)
            aa=clist.index(line)
            #print(aa)
            if(dlist[aa]==2):
                print(line)
    
        

In [ ]:
with open(os.path.join("/project/train/cvmart/dataset", 'label_map.pbtxt'), 'w') as f:
  label_map = """
  item {
  id: 1
  name: "lifevest_person"
  },
  item {
  id: 2
  name: "no_lifevest_person"
  }
  """
  f.write(label_map)

In [ ]:
lowstr="/project/train/NJSK/NJSK_lifevest_none_none_none_train_none_day_20210415_65.xml"
num_str=lowstr[0:-4]
print(num_str)